# Thema 3 werkcollege 1: Word Embeddings
In dit werkcollege gaan we aan de slag met Word embeddings. Op de elo vind je een bestand `gamereviews.xlsx`met daarin 100.000 videogames reviews van Amazon (Orginenele bestand http://jmcauley.ucsd.edu/data/amazon/index_2014.html). Voor iedere revuew is het `sentiment` (1 positief; 0 negatief) bepaald op het aantal gegeven sterren (1-2 negatief; 4-5 positief) Verder bevat het bestand een kolom `reviewText` met daarin de daadwerkelijke review. We gaan proberen om op basis de text proberen te voorspellen of een review positief of negatief is. 
- lees het bestand in
- maak een variabele `text`aan met daarin de reviews als string
- maak een variabele `sentiment` aan met daarin een 1 als er 4 of 5 sterren gegeven zijn en een 0 als er 1 of 2 sterren gegeven zijn. Geef deze als tyoe int 

In [ ]:
# lees het bestand gamereviews in in een pandas dataframe 
# Split deze in een variabele text en een variabele sentiment met daarin de sentimenten 


We hebben nu 100.000 reviews, waarvan er 50.000 positief zijn. We gaan nu de tokenizen zodat ieder woord een eigen integer waarde krijgt. 

In [ ]:
#tokenize de tekst ga uit van max 10.000 woorden

Nu hebben we eerder al gezien dat niet alle reviews even lang zijn. Daarom gaan we deze aanvullen (padding) naar 50 woorden. Lange reviews worden afgekapt. Ivm de performance beperken we ons hier tot de 1e 50 woorden.

In [ ]:
#vul de reviews aan tot 50 woorden

We gaan nu sequentieel model bouwen en fitten. Het model bestaat uit een embedding layer, een flatten layer en sluit af met een Dense layer met 1 neuron. De embedding layer geven we 10.000 woorden als max en 100 dimensies. 

In [ ]:
#bouw een model met een embedding layer 

In [ ]:
#compile en fit het model. epochs=10 validation_split=0.2

In [ ]:
#plot de accuracy en validated accuracy

Wat valt hier op? 

We testen ons model ook even met een paar eigenbedachte tweets. Schrijf hiervoor de code

In [ ]:
#test een zelfbedachte tweet. Zorg ervoor dat er negative of positive uitkomt

Vaak heb je onvoldoende data om zelf alles te trainen. Gelukkig is dit voor Engels al veelvuldig gedaan. Een bekend voorbeeld hiervan is GloVe (Global Vectors for words). We beginnen daarom met het downloaden van Glove [https://nlp.stanford.edu/projects/glove/] en download de 2014 English Wikipedia. Dit is een zipfile met daarin 100-dimensionale vectors voor 400.000 engelse woorden.

In [ ]:
#download glove. Pas het path aan en run de onderstaande code
import os
import numpy as np
glove_dir = 'C:\\Detail opzet Text\\GloVe\\'


embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
print(embeddings_index['the'])  #voorbeeld van een willekeurig woord

Als het goed is zijn er nu 400000 vectors ingelezen. De volgende stap is het omzetten van de variabele embedding_index in een matrix die we aan een embedding layer kunnen voeren. Deze moet de vorm max_features, embedding_dim hebben. In dit geval dus 20.000 woorden en 100 dimensies.  

In [ ]:
#Bestudeer, begrijp en run de onderstaande code
embedding_dim = 100

embedding_matrix = np.zeros((max_features, embedding_dim)) #initialiseren alles op 0
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_features:
        if embedding_vector is not None:   # als het woord gevonden is
            embedding_matrix[i] = embedding_vector
            
embedding_matrix[3] #voorbeeldje

Ons model blijft gelijk met eerder. Dus hier hoeven we niks aan te veranderen

We laden onze variabele embedding_matrix in als <u>weight</u> (tip: gebruik set_weights) bij de embedding layer [0].
Aangezien we de bestaande glove matrix willen gebruiken en niet opnieuw gaan trainen zetten we trainable op False

In [ ]:
#gebruik de embedding_matrix als weigth bij de embed_layer en set trainable of false

We compilen en fitten het model opnieuw.

In [ ]:
#compile en fit het model

In [ ]:
#plot de accuracy en validated accuracy

Wat valt hier op?

Hieronder volgt nog een leuk stukje visualisatie van wat GloVe precies allemaal doet

We beginnen met een lijstje met redelijk willekeurige woorden die we opzoeken in de GloVe dictionary. Dit geeft ons per word 50 verschilende waarden. Nu is een 50-dimensionale figuur vrij lastig te visualiseren, daarom reduceren we dit door middel van Principal Component Analysis tot 2 (zie https://en.wikipedia.org/wiki/Principal_component_analysis) en plotten we deze.

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from plotnine import *

words = ['horse', 'cow', 'monkey', 'cat', 'apple', 'banana', 'pear', 'mango',
         'car', 'bus', 'truck', 'bike', 'computer', 'calculator', 'keyboard', 'mouse']

plot_word = pd.DataFrame(PCA(n_components=2).fit_transform([embeddings_index[w] for w in words]), 
                         columns=['PC1', 'PC2'])

plot_word['word'] = words
plot_word['type'] = list(pd.Series(['animal', 'fruit', 'transport', 'PC stuff']).repeat(4))

gg = (
    ggplot(plot_word, aes(x='PC1', y='PC2')) + 
      geom_point(aes(color='type')) + 
      geom_label(aes(label='word', color='type'))
)
gg

Wat hier opvalt is dat veel vervante woorden (zoals bijvoorbeeld de voertuigen) allemaal redelijk dicht bij elkaar liggen. Wat ook opvalt is dat 'apple' tussen de fruitsoorten en de computer onderdelen is geplaatst. Hetzelde geldt ook enigszins voor 'mouse'

Nu is zijn niet alleen losse woorden heel interessant, maar ook hoe woorden zich tot elkaar verhouden. Daarom plotten we in dit geval een aantal landen en hoofdsteden. wat hierbij opvalt is dat dat tussen liggende vector (verplaatsing op de x en de y-as) bij veel landen min of meer gelijk is.

In [ ]:
country_capitals = ['china', 'bejing', 'russia', 'moscow', 'japan', 'tokyo', 
                    'turkey', 'ankara', 'spain', 'madrid', 'france', 'paris',
                    'portugal', 'lisbon', 'greece', 'athens', 'italy', 'rome',
                    'netherlands', 'amsterdam', 'germany', 'berlin']
plot_word = pd.DataFrame(PCA(n_components=2).fit_transform([embeddings_index[w] for w in country_capitals]), 
                         columns=['PC1', 'PC2'])
plot_word['word'] = country_capitals
plot_word['concept'] = pd.Series(['country', 'capital'] * int(len(plot_word) / 2))
plot_word['country'] = list(pd.Series(['china', 'russia', 'japan', 'turkey', 'spain', 'france', 'portugal', 'greece', 'italy', 'netherlands', 'germany']).repeat(2))
plot_word = plot_word.pivot(index='country', columns='concept')  # Needed for plotting in plotnine
plot_word.columns = ['_'.join(col).strip() for col in plot_word.columns.values]

gg = (
    ggplot(plot_word) + 
      geom_point(aes(x='PC1_country', y='PC2_country'), color='blue') + 
      geom_text(aes(x='PC1_country', y='PC2_country', label='word_country'), ha='left') + 
      geom_point(aes(x='PC1_capital', y='PC2_capital'), color='green') + 
      geom_text(aes(x='PC1_capital', y='PC2_capital', label='word_capital'), ha='right') +
      geom_segment(aes(x = 'PC1_country', xend='PC1_capital', y='PC2_country', yend='PC2_capital'))
)
gg

Dit gaat zelfs zo ver dat we dat je hier min-of-meer mee kunt rekenen. Hieronder zie je wat er gebeurt als je 'paris' - 'france' + 'poland' uitrekend en dan als uitkomst het dichtsbijzijnde woord pakt. 

In [ ]:
from scipy.spatial import cKDTree

# Represent the embeddings as a matrix, needed for search with cKDTree
word_indices = pd.Series(list(embeddings_index.keys()))
cKDTree_embedding_matrix = np.array(list(embeddings_index.values()))

# Perform some vector magic with embeddings
find_capital_poland = embeddings_index['paris'] - embeddings_index['france'] + embeddings_index['poland']
closest_word = cKDTree(cKDTree_embedding_matrix).query(find_capital_poland, k=1) #zoek de k dichtsbijzijnde neighbors op
word_indices[closest_word[1]]

# Thema 3 werkcollege 2: Recurrent Neural Networks
Genoeg spielerij met word_embeddings en terug naar onze videogames dataset. Nu we onze data in een bruikbaar formaat hebben kunnen we ons netwerk verder gaan opbouwen en trainen. We beginnen met het toevoegen van onze 1e SimpleRNN layer.

In [ ]:
#vervang de flatten layer met een SimpleRNN layer met 32 nodes

In [ ]:
#compile en fit epochs = 3

In [ ]:
#plot de accuracy en validated accuracy

Wat valt hier op?

Eventueel kun je het netwerk nog wat dieper trainer door er 1 of meerdere extra SimpleRNN layers aan toe te voegen. Gezien de tijd die het trainen van dit netwerk zou kosten gaan we dat nu even niet doen. Mocht je het wel thuis willen proberen, let er dan op dat je de juiste output 'returned'.

Nu is een SimpleRNN eigenlijk wat te simpel om de complexiteit van tekst te vatten, daarom vervangen we onze simpleRNN_layer door een LSTM layer met voor het gemak ook maar 32 neurons.

In [ ]:
#vervang de simpleRNN_layer door een LSTM_layer met 32 neurons epochs=10

In [ ]:
#plot de accuracy en validated accuracy

Wat valt hier op?

We proberen het nog eens met een 2e LSTM-laag ook met 32 neurons (let weer op de return sequence).

In [ ]:
#voeg een 2e LSTM laag toe, compile, fit en plot de (validated) accuracy 

Voegt dit veel toe? 

Een alternatief voor LSTM-layers zijn GRU-layer, vervang daarom de LSTM-layers door GRU-layers.

In [ ]:
#Vervang de LSTM-layers door GRU-layers, compile, fit en plot de (validated) accuracy 

Wat zijn de verschillen t.o.v. ons model met LSTM layer? 

Aangezien ons model nog steeds overfit voegen we aan de 1e GRU-layer een dropout en een recurrent dropout van 0.2 toe. Ivm performance beperken we ons tot 2 epochs (het punt waar de vorige ging overfitten) en forceren we het model op de cpu te draaien. 

In [ ]:
#Voeg aan de 1e GRU layer een dropout en recurrentdropout van 0.2 toe, compile fit en plot
#NB: beperkt je ivm de performance tot 2 epochs en forceer op cpu
# with tf.device('/cpu:0'):
#    history = model.fit(.....)



Wat zien we gebeuren?

Een andere optie om te verkennen is het bi-directioneel maken van 1 of beide GRU layers. Ivm de performance schrappen we de recurrent_dropout (de dropout blijft).

In [ ]:
#maak beide GRU-layers bi-directional, schrap de recurrent_dropout. Compile, fit (10 epochs) en plot de (validated) accuracy.

Hoeveel effect heeft dit?

Tot slot kijken we naar het effect als we onze GRU-layers vervangen door conv1d_layers, met bijbehorende maxpooling 1d en flatten layers

In [ ]:
#Vervang beide GRU layers door Conv1D_layers, een MaxPooling1d_layer, en een flatten_layer. Compile, Fit en Plot

Hoeveel effect heeft dit? 

Wat kun je zeggen over het gebruik van RNN-layers voor sentiment-analyse?

In dit geval lijken de RNN layers een beperkt effect te hebben. Bij heel kleine datasets is het zo dat een bag-of-words benadering zoals met alleen een embedding het vaak beter doen RNN's. De belangrijkste oorzaak zit hem in <u>de hoeveelheid samples t.o.v. de gemiddelde sample lengte </u> (hier afgekapt op 100).  Een richtlijn uit cholet p349 is dat als het <u> aantal samples gedeeld door de gemiddelde sample length < 1500</u> een bag-of-words methode beter werkt dan een transformer of een RNN. In deze casus trainen we met 90.000 reviews en 50 woorden = 1800 waardoor we nog maar heel beperkt boven deze vuistregel zitten